# google drive mount

In [1]:
from google.colab import drive
drive.mount('/content/data')

Mounted at /content/data


# 이미지 전처리와 데이터 split

In [2]:
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets
import os
import pandas as pd

# 데이터 증강

In [3]:
# 0. import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os, glob
import random
from tqdm import tqdm
from time import time

In [ ]:
# 1. 데이터 증강에 사용할 함수정의
def contrast(gray, min_val, max_val):
    #gray = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    alpha = int(random.uniform(min_val, max_val)) # Contrast control
    adjusted = cv2.convertScaleAbs(gray, alpha=alpha)
    return adjusted

def fill(img, h, w):
    img = cv2.resize(img, (h, w), cv2.INTER_CUBIC)
    return img

def rotation(img, angle):
    # angle = int(random.uniform(-angle, angle))
    angle_list = [int(angle), 15, 25, 90, 45,90,180, 180] #조절
    angle = random.choice(angle_list)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img


def vertical_shift_down(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    img = fill(img, h, w)
    return img


def vertical_shift_up(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(0.0, ratio)
    h, w = img.shape[:2]
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :, :]
    img = fill(img, h, w)
    return img

def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < 0:
        print('Value should be less than 1 and greater than 0')
        return img
    ratio = random.uniform(-ratio, ratio)
    h, w = img.shape[:2]
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift), :]
    if ratio < 0:
        img = img[:, int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img


def vertical_flip(img, flag):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

def horizontal_flip(img, flag):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

## 데이터 불균형 해결 
> lable당 100개의 이미지로 증량

In [4]:
PATH = '/content/data/MyDrive/deeplearning/train_new/'

image_dirs = os.listdir(PATH)
image_paths = dict()
for image_dir in tqdm(image_dirs):
    try:
        files = glob.glob(PATH + image_dir+"/*.png") 
        image_paths[image_dir] = [file for file in files]
    except:
        FileNotFoundError

100%|██████████| 88/88 [00:16<00:00,  5.22it/s]


In [5]:
# for문으로 이미지 파일, path 가져와서 data에에 filename, label 입력
import PIL
from PIL import Image
import torch
import torchvision
from torch.utils.data import  TensorDataset, DataLoader

data = dict()
data['label'] = []
data['filename'] = []
# data['image_arr'] = []
for label,paths in tqdm(image_paths.items()):
    for path in paths:
        img = Image.open(path)
        data['filename'].append(path)
        # data['image_arr'].append(img_arr)
        data['label'].append(label)

100%|██████████| 88/88 [40:05<00:00, 27.34s/it]


In [6]:
from collections import Counter
import matplotlib.pyplot as plt
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array,load_img
#print('number of samples: ', len(data['image_arr']))
print('dict keys: ', list(data.keys()))
# print('image shape: ', data['image_arr'][0].shape)
count_label = Counter(data['label'])
print('Original dataset shape')
print(sorted(count_label.items()))

dict keys:  ['label', 'filename']
Original dataset shape
[('bottle-broken_large', 100), ('bottle-broken_small', 100), ('bottle-contamination', 100), ('bottle-good', 209), ('cable-bent_wire', 100), ('cable-cable_swap', 100), ('cable-combined', 100), ('cable-cut_inner_insulation', 100), ('cable-cut_outer_insulation', 100), ('cable-good', 224), ('cable-missing_cable', 100), ('cable-missing_wire', 100), ('cable-poke_insulation', 100), ('capsule-crack', 100), ('capsule-faulty_imprint', 100), ('capsule-good', 219), ('capsule-poke', 100), ('capsule-scratch', 100), ('capsule-squeeze', 100), ('carpet-color', 100), ('carpet-cut', 100), ('carpet-good', 280), ('carpet-hole', 100), ('carpet-metal_contamination', 100), ('carpet-thread', 100), ('grid-bent', 100), ('grid-broken', 100), ('grid-glue', 100), ('grid-good', 264), ('grid-metal_contamination', 100), ('grid-thread', 100), ('hazelnut-crack', 100), ('hazelnut-cut', 100), ('hazelnut-good', 391), ('hazelnut-hole', 100), ('hazelnut-print', 100), (

In [7]:
PATH = '/content/data/MyDrive/deeplearning/train_new/'
image_dirs = os.listdir(PATH)
len_image_list = 0

label_count = Counter(data['label'])
label_max = max(label_count.values())


for image_dir in tqdm(image_dirs):
    image_list = os.listdir(PATH+image_dir+"/")
    len_image_list = len(image_list)

    while(len_image_list<100):   # 폴더에 생성할 갯수 설정
        
        for image in image_list:
            img = cv2.imread(PATH+image_dir+"/"+image)
            img = contrast(img, 1, 1.5)
            img = horizontal_flip(img, 1)
            img = rotation(img, 45)
            img = horizontal_shift(img, 0.1)
            file_name = PATH+image_dir+"/arumentation_"+str(time())+ '.png'
            cv2.imwrite(file_name, img)
            len_image_list += 1
            if len_image_list >= 100:
                break;

    len_image_list = 0

100%|██████████| 88/88 [00:00<00:00, 244.09it/s]


## 데이터 전처리

In [8]:
pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.6 MB/s eta 0:00:00


In [9]:
pip install GPUtil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=658631a05901b72a485cf88f672aca51a1ef76951c11cad5344efe974d9e4faa
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built GPUtil


In [10]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data.sampler import SubsetRandomSampler
import time


#CUDA 메모리 확인, 상황에 맞게 설정할 것 
import GPUtil
# GPUtil.showUtilization()

# GPU 사용 설정과 random seed 설정
print('pytorch version: {}'.format(torch.__version__)) 
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available())) 
device = "cuda:0" if torch.cuda.is_available() else "cpu" 



#device = torch.device("cpu")



# 일관된 weight initialization을 위한 random seed 설정
torch.manual_seed(777) 
if device == "cuda:0":
    torch.cuda.manual_seed_all(777)

pytorch version: 1.13.1+cu116
GPU 사용 가능 여부: True


In [ ]:
"""
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

train_datasets = datasets.ImageFolder(os.path.join('/content/data/MyDrive/deeplearning/train_new/'), transform )

print("Train 데이터 모양", train_datasets)
"""

"""
meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_datasets]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_datasets]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])
"""
# 512 x 512
# meanR, meanG, meanB, stdR, stdG, stdB
# ([0.41017398,0.38485712,0.37689483], [0.20070569,0.19670747, 0.18732636])


'\nmeanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_datasets]\nstdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_datasets]\n\nmeanR = np.mean([m[0] for m in meanRGB])\nmeanG = np.mean([m[1] for m in meanRGB])\nmeanB = np.mean([m[2] for m in meanRGB])\n\nstdR = np.mean([s[0] for s in stdRGB])\nstdG = np.mean([s[1] for s in stdRGB])\nstdB = np.mean([s[2] for s in stdRGB])\n'

## 데이터셋 증강

In [11]:
batch_size = 64

In [12]:
data_dir = "/content/data/MyDrive/deeplearning/train_new/"

def create_datasets(batch_size):
    
    train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 좌우반전 
    transforms.RandomVerticalFlip(),  # 상하반전 
    transforms.Resize((512, 512)),  # input size
    transforms.ToTensor(),  # 이 과정에서 [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize([0.41017398,0.38485712,0.37689483], [0.20070569,0.19670747,0.18732636])  #  정규화(normalization)
])

    # choose the training
    train_data = datasets.ImageFolder(os.path.join(data_dir), train_transform)

    # trainning set 중 validation 데이터로 사용할 비율
    valid_size = 0.3

    # validation으로 사용할 trainning indices를 얻는다.
    num_train = len(train_data)
    indices = list(range(num_train))
    np.random.shuffle(indices)
    split = int(np.floor(valid_size * num_train))
    train_idx, valid_idx = indices[split:], indices[:split]

    # trainning, validation batch를 얻기 위한 sampler정의
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    # load training data in batches
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=train_sampler,
                                               num_workers=4)

    # load validation data in batches
    valid_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               sampler=valid_sampler,
                                               num_workers=4)
    

    return train_data, train_loader, valid_loader


train_data, train_loader, valid_loader = create_datasets(batch_size)

In [14]:
test_png = sorted(glob('/content/data/MyDrive/deeplearning/test/test/*.png'))

train_y = pd.read_csv("/content/data/MyDrive/deeplearning/train_df.csv")
train_labels = train_y["label"]
label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

def img_load(path):
    img = cv2.imread(path)[:,:,::-1] # opencv는 B,G,R 이기 때문에 R,G,B로 변경해주는 것 
    img = cv2.resize(img, (512, 512))
    return img # type = numpy.ndarray

test_imgs = [img_load(n) for n in tqdm(test_png)]


100%|██████████| 2154/2154 [02:30<00:00, 14.32it/s]


In [15]:
import torch.nn.functional as F

class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, transforms):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        img = self.transforms(img)   #   train이든 test든 totensor 로 0~1로 nomaliztion
        label = self.labels[idx]
        
        return img, label

In [16]:
test_transform = transforms.Compose([
    transforms.ToTensor(),  # [0, 255]의 범위를 갖는 값들을 [0.0, 1.0]으로 정규화, torch.FloatTensor로 변환
    transforms.Normalize([0.41017398,0.38485712,0.37689483], [0.20070569,0.19670747,0.18732636])   #  정규화(normalization)
])

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), transforms=test_transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# 모델링

In [17]:
n_devices = torch.cuda.device_count()
# 사용 가능한 CUDA 인덱스 출력
for i in range(n_devices):
  print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Device 0: Tesla T4


## 모델 정의

In [18]:
import torch
device = torch.device('cuda:0')

## efficientnet_b1 모델

In [19]:

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b1', pretrained=True, num_classes=88) # 원하는 모델 선택
           
    def forward(self, x):
        x = self.model(x)
        # x = F.softmax(x,dim=1)
        return x

model = Network().to(device)


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b1-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-533bc792.pth


## 앙상블 모델
> efficientnet_b7, resnet50, vit_base_patch16_384

In [ ]:
"""
import torch
import torch.nn as nn
import timm

class Ensemble(nn.Module):
    def __init__(self):
        super(Ensemble, self).__init__()
        self.model1 = timm.create_model('efficientnet_b7', pretrained=True, num_classes=88)
        self.model2 = timm.create_model('resnet50', pretrained=True, num_classes=88)
        self.model3 = timm.create_model('vit_base_patch16_384', pretrained=True, num_classes=88)
        
    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        out3 = self.model3(x)
        
        out = (out1 + out2 + out3) / 3
        out = torch.softmax(out, dim=1)
        return out

model = Ensemble().to(device)
"""

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth" to /root/.cache/torch/hub/checkpoints/resnet50_a1_0-14fe96d1.pth


## 모델 학습

### 전이 학습

In [20]:
model.load_state_dict(torch.load('/content/data/MyDrive/deeplearning/0.9921008917114432.pt'))

<All keys matched successfully>

### 학습 및 평가

In [21]:
epochs = 30

#### 평가 산식 : Macro F1 Score

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score


# weight_decay
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001 )
# 학습률 스케쥴러 추가
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 10, gamma = 0.01)

criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler()    # amp 학습 빠르게

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            
            pred = model(x)     
                
        loss = criterion(pred, y)    
       
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    scheduler.step()
           
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    if train_f1 > 0.90:
        torch.save(model.state_dict(), f"/content/data/MyDrive/deeplearning/{train_f1}.pt")

# 결과 제출

## 저장된 모델 중에서 골라오기

In [23]:
model.load_state_dict(torch.load('/content/data/MyDrive/deeplearning/0.9921008917114432.pt'))

<All keys matched successfully>

## submmission 생성

In [24]:
model.eval()
f_pred = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [25]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [26]:
submission = pd.read_csv("/content/data/MyDrive/deeplearning/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-glue
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-bent
2152,2152,cable-poke_insulation


In [27]:
submission.to_csv("0320_b1.csv", index = False)